In [ ]:
import time
import re
import requests
import os
os.environ["CURL_CA_BUNDLE"]=''
import sys
import urllib3
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import shutil

def is_image_url(url):
    response = requests.head(url)
    content_type = response.headers.get('Content-Type')
    if content_type and content_type.startswith('image/'):
        return True
    return False

def deal_md(md_file_name, origin_md_path, output_md_path, image_dir, image_url_prefix):
    
    idx = 0
    output_content = []
    image_file_prefix = md_file_name.replace('.md','') + '-'
    with open(origin_md_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():             
            if ('http' in line) and ('.png'in line or '.jpeg' in line or '.jpg' in line or '.gif' in line or '.webp' in line
                    or '.xbm' in line or '.tif' in line or '.pjp' in line or '.svgz' in line or '.ico' in line or '.tiff' in line
                    or '.svg' in line or '.jfif' in line or '.bmp' in line or '.pjpeg' in line or '.avif' in line):
                
                if '.png' in line:
                    suffix = '.png'
                    line = re.sub(r'.png#(.*)+', '.png)', line)
                elif '.jpeg' in line:
                    suffix = '.jpeg'
                    line = re.sub(r'.jpeg#(.*)+', '.jpeg)', line)
                elif '.jpg' in line:
                    suffix = '.jpg'
                    line = re.sub(r'.jpg#(.*)+', '.jpg)', line)
                elif '.gif' in line:
                    suffix = '.gif'
                    line = re.sub(r'.gif#(.*)+', '.gif)', line)
                elif '.webp' in line:
                    suffix = '.webp'
                    line = re.sub(r'.webp#(.*)+', '.webp)', line) 
                elif '.xbm' in line:
                    suffix = '.xbm'
                    line = re.sub(r'.xbm#(.*)+', '.xbm)', line)
                elif '.tif' in line:
                    suffix = '.tif'
                    line = re.sub(r'.tif#(.*)+', '.tif)', line)
                elif '.pjp' in line:
                    suffix = '.pjp'
                    line = re.sub(r'.pjp#(.*)+', '.pjp)', line)
                elif '.svgz' in line:
                    suffix = '.svgz'
                    line = re.sub(r'.svgz#(.*)+', '.svgz)', line)
                elif '.ico' in line:
                    suffix = '.ico'
                    line = re.sub(r'.ico#(.*)+', '.ico)', line) 
                elif '.tiff' in line:
                    suffix = '.tiff'
                    line = re.sub(r'.tiff#(.*)+', '.tiff)', line)
                elif '.svg' in line:
                    suffix = '.svg'
                    line = re.sub(r'.svg#(.*)+', '.svg)', line)
                elif '.jfif' in line:
                    suffix = '.jfif'
                    line = re.sub(r'.jfif#(.*)+', '.jfif)', line)
                elif '.bmp' in line:
                    suffix = '.bmp'
                    line = re.sub(r'.bmp#(.*)+', '.bmp)', line)
                elif '.pjpeg' in line:
                    suffix = '.pjpeg'
                    line = re.sub(r'.pjpeg#(.*)+', '.pjpeg)', line) 
                elif '.avif' in line:
                    suffix = '.avif'
                    line = re.sub(r'.avif#(.*)+', '.avif)', line)
                
                image_url = str(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', line))
                image_url = image_url.replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace("'", '')
                if(is_image_url(image_url)) :                   
                    download_image(image_url, image_dir, image_file_prefix, idx+1, suffix)
                    to_replace = '/'.join(image_url.split('/')[:-1])
                    new_image_url = image_url.replace(to_replace, 'placeholder')                    
                    new_image_url = image_url_prefix + image_file_prefix + str(idx+1) + suffix                    
                    idx += 1
                    line = line.replace(image_url, new_image_url)
            output_content.append(line)
    with open(output_md_path, 'w', encoding='utf-8', errors='ignore') as f:
        for _output_content in output_content:
            f.write(str(_output_content))
    return idx


def download_image(image_url, image_dir, image_file_prefix, idx, suffix):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    r = requests.get(image_url, stream=True, verify=False)
    image_name = image_url.split('/')[-1]
    image_name = image_file_prefix + str(idx) + suffix
    if r.status_code == 200:
        open(image_dir + '/' + image_name, 'wb').write(r.content)
    r.close()


def mkdir(output_md_path, image_url_prefix):
    image_dir = output_md_path + image_url_prefix
    image_dir = image_dir.strip()
    image_dir = image_dir.rstrip("\\")
    isExists = os.path.exists(image_dir)
    if isExists:        
        shutil.rmtree(output_md_path)
    os.makedirs(image_dir)
    return image_dir

def main():
    '''
    Note:执行前请关掉vpn
    按需更改以下2个路径后，执行代码
    origin_md_path: 待处理的markdown文件路径
    output_md_path: 处理后的markdown文件路径，此路径无须创建，代码会进行创建
    '''
    origin_md_path = 'E:/workspace/ju/old/'
    output_md_path = 'E:/workspace/ju/new/'
    image_url_prefix = 'images/'    
    image_dir = mkdir(output_md_path, image_url_prefix)
    fileList = os.listdir(origin_md_path)
    md_sum = 0;
    pic_sum = 0;
    fail = 0;
    for f in fileList:
        if len(f) >= 3 and f[-3:] == '.md':
            try:
                md_sum += 1;
                cnt = deal_md(f, origin_md_path + f, output_md_path + f, image_dir, image_url_prefix)
                pic_sum += cnt;
                print('处理完成第{}个文件：{}, 共{}张图片'.format(md_sum,f,cnt))
                time.sleep(1)
            except Exception as e:
                fail += 1;
                print('文件:{}报错，异常信息:{}'.format(f,e))
    print('全部处理完成，文件总数：{}, 成功文件数：{}，成功图片数：{}，失败文件数：{}'.format(md_sum,md_sum-fail,pic_sum,fail))

if __name__ == '__main__':
    main()